In [1]:
from pymongo import MongoClient
import pandas as pd
from getpass import getpass
from dotenv import load_dotenv
import os
import requests
import json
import plotly.graph_objects as go
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster


In [2]:

load_dotenv()
token = os.getenv('token')

In [3]:
client = MongoClient("localhost:27017")
db = client['Ironhack']
c = db.get_collection("companies")

In [4]:
c.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

In [5]:
condition1 = {"category_code":"games_video"}
condition2 = {"funding_rounds.raised_amount":{"$gt": 1000000}}
query = {"$and":[condition1,condition2]}
projection = {"name":1, "_id":0, "offices.city":1}
games = list(c.find(query, projection))
df = pd.DataFrame(games)
df = df.explode('offices')
df.offices.value_counts()

offices
{'city': 'San Francisco'}    39
{'city': 'New York'}         30
{'city': 'Los Angeles'}      11
{'city': ''}                  8
{'city': 'Paris'}             8
                             ..
{'city': 'Boulder'}           1
{'city': 'Dundee'}            1
{'city': 'TromsÃ¸'}           1
{'city': 'Abercynon'}         1
{'city': 'Vancouver'}         1
Name: count, Length: 118, dtype: int64

In [6]:
condition1 = {"category_code":"web"}
condition2 = {"funding_rounds.raised_amount":{"$gt": 1000000}}
query = {"$and":[condition1,condition2]}
projection = {"name":1, "_id":0, "offices.city":1}
web = list(c.find(query, projection))
df = pd.DataFrame(web)
df = df.explode('offices')
df.offices.value_counts()


offices
{'city': 'San Francisco'}    71
{'city': 'New York'}         65
{'city': 'Palo Alto'}        22
{'city': 'Seattle'}          20
{'city': 'London'}           20
                             ..
{'city': 'Waltham'}           1
{'city': 'Pittsburgh'}        1
{'city': 'Burlingame'}        1
{'city': 'Lowell'}            1
{'city': 'Calabassas'}        1
Name: count, Length: 194, dtype: int64

## SAN FRANCISCO was the chosen location to steal a company's venue.

In [7]:
# Getting all ofices in San Francisco
condition1 = {"number_of_employees":{"$lt": 120}}
condition2 = {"number_of_employees":{"$gte": 90}}
condition3 = {"offices.city":"San Francisco"}
query = {"$and":[condition1,condition2,condition3]}

pipeline = [
    {"$match": query},
    {"$unwind": "$offices"},
    {"$match": {"offices.city": "San Francisco"}},
    {"$project": {"_id": 0, "name": 1, "latitude": "$offices.latitude", "longitude": "$offices.longitude", "address": "$offices.address1"}}
]

final_list = list(c.aggregate(pipeline))
final_list

[{'name': 'hi5',
  'latitude': 37.788668,
  'longitude': -122.400558,
  'address': '55 Second Street'},
 {'name': 'eBuddy',
  'latitude': 37.787683,
  'longitude': -122.410943,
  'address': '555 Post St.'},
 {'name': 'Greystripe',
  'latitude': 37.710677,
  'longitude': -122.39323,
  'address': '160 Spear Street Fl 14'},
 {'name': 'Zynga',
  'latitude': 37.765158,
  'longitude': -122.404234,
  'address': '365 Vermont St.'},
 {'name': 'Smaato',
  'latitude': None,
  'longitude': None,
  'address': '240 Stockton St, 10th Floor'},
 {'name': 'AtomicOnline',
  'latitude': None,
  'longitude': None,
  'address': '244 Jackson Street'},
 {'name': '99designs',
  'latitude': 37.7955307,
  'longitude': -122.4005983,
  'address': '447 Battery St.'},
 {'name': 'LiveRail',
  'latitude': 37.787183,
  'longitude': -122.397759,
  'address': '575 Market St.'},
 {'name': 'Clickpass',
  'latitude': 37.775196,
  'longitude': -122.419204,
  'address': ''},
 {'name': 'Pure Digital Technologies',
  'latitude'

In [8]:
possible_venues = pd.DataFrame(final_list)
possible_venues.drop_duplicates(inplace=True)
possible_venues.dropna(inplace=True)
possible_venues.head()

,name,latitude,longitude,address
0,hi5,37.788668,-122.400558,55 Second Street
1,eBuddy,37.787683,-122.410943,555 Post St.
2,Greystripe,37.710677,-122.393230,160 Spear Street Fl 14
3,Zynga,37.765158,-122.404234,365 Vermont St.
6,99designs,37.795531,-122.400598,447 Battery St.


In [9]:
map_venues = px.scatter_mapbox(possible_venues, lat="latitude", lon="longitude", hover_name="name", hover_data=["address",])
map_venues.update_layout(mapbox_style="carto-positron",mapbox_zoom=12)
pio.write_html(map_venues, file="image/possible_venues.html", auto_open=True)

In [10]:
import folium

map_venues2 = folium.Map(location=[possible_venues.loc[0, 'latitude'], possible_venues.loc[0, 'longitude']], zoom_start=13,tiles='cartodbpositron')

# iterate over the rows in the DataFrame and add a marker for each location
for index, row in possible_venues.iterrows():
    tooltip = f"{row['name']}: {row['address']}"
    icon = Icon (
        color = "blue",
        icon_color = "white",
        icon = "building ",
        prefix = "fa")
    folium.Marker([row['latitude'], row['longitude']], tooltip=tooltip, icon=icon).add_to(map_venues2)

# display the map
map_venues2

In [11]:
def foursquare_places (venue, coordinates):
    """
    This function get resu
    args:
    :venue: String for the type of business you are searching for
    :coordinates: String for the region you want the search to be executed
    returns 
    """
    url = f"https://api.foursquare.com/v3/places/search?query={venue}&near={coordinates}&limit=60"
    headers = {"accept": "application/json", "Authorization": token}
    retorno = requests.get(url, headers=headers).json()

    places = []
    
    for result in retorno["results"]:
        name = result["name"]
        lat = result["geocodes"]["main"]["latitude"]
        lon = result["geocodes"]["main"]["longitude"]
        places.append({"name": name, "lat": lat, "lon": lon})
    
    return places

In [12]:
"""
starbucks = foursquare_places ("starbucks", "San Francisco")
starbucks
"""

'\nstarbucks = foursquare_places ("starbucks", "San Francisco")\nstarbucks\n'

In [13]:
"""
with open("data/starbucks.json", "w") as f:
    json.dump(starbucks, f)
"""

'\nwith open("data/starbucks.json", "w") as f:\n    json.dump(starbucks, f)\n'

In [14]:
with open("data/starbucks.json", "r") as f:
    starbucks_venues = json.load(f)


In [15]:
"""
bars = foursquare_places ("bars", "San Francisco")
with open("data/bars.json", "w") as f:
    json.dump(bars, f)
bars
"""
with open("data/bars.json", "r") as f:
    bar_venues = json.load(f)

In [16]:

vegan = foursquare_places ("vegan", "San Francisco")
with open("data/vegan.json", "w") as f:
    json.dump(vegan, f)

#with open("data/vegan.json", "r") as f:
#    vegan_venues = json.load(f)

KeyError: 'results'

In [ ]:
"""
Daycare = foursquare_places ("Daycare", "San Francisco")
with open("data/Daycare.json", "w") as f:
    json.dump(Daycare, f)
"""
with open("data/daycare.json", "r") as f:
    daycare_venues = json.load(f)

In [ ]:
"""
pet = foursquare_places ("Pet Grooming Service", "San Francisco")
with open("data/pet.json", "w") as f:
    json.dump(pet, f)
"""
with open("data/pet.json", "r") as f:
    grooming_venues = json.load(f)

In [ ]:
map_alltogether = folium.Map(location=[possible_venues.loc[0, 'latitude'], possible_venues.loc[0, 'longitude']], zoom_start=13,tiles='cartodbpositron')

# create a list of (latitude, longitude) tuples from your DataFrame
locations = [(venue['lat'], venue['lon']) for venue in grooming_venues]

# add a heatmap layer to the map
heatmap_layer = HeatMap(locations)
heatmap_layer.add_to(map_alltogether)

# iterate over the rows in the DataFrame and add a marker for each location
for index, row in possible_venues.iterrows():
    tooltip = f"{row['name']}: {row['address']}"
    icon = folium.Icon (
        color = "blue",
        icon_color = "white",
        icon = "building ",
        prefix = "fa")
    folium.Marker([row['latitude'], row['longitude']], tooltip=tooltip, icon=icon).add_to(map_alltogether)

# display the map
map_alltogether

In [ ]:
# create a list of (latitude, longitude) tuples from the grooming_venues list
grooming_locations = [(venue['lat'], venue['lon']) for venue in grooming_venues]

# create a heatmap layer with the grooming locations and add it to the map
grooming_heatmap = HeatMap(grooming_locations, name='Pet Grooming')
grooming_heatmap.add_to(map_venues2)

# create a list of (latitude, longitude) tuples from the starbucks_venues list
starbucks_locations = [(venue['lat'], venue['lon']) for venue in starbucks_venues]

# create a heatmap layer with the Starbucks locations and add it to the map
starbucks_heatmap = HeatMap(starbucks_locations, name='Starbucks')
starbucks_heatmap.add_to(map_venues2)

# create a list of (latitude, longitude) tuples from the starbucks_venues list
daycare_locations = [(venue['lat'], venue['lon']) for venue in daycare_venues]

# create a heatmap layer with the Starbucks locations and add it to the map
daycare_heatmap = HeatMap(daycare_locations, name='Daycare')
daycare_heatmap.add_to(map_venues2)

# iterate over the rows in the DataFrame and add a marker for each location
for index, row in possible_venues.iterrows():
    tooltip = f"{row['name']}: {row['address']}"
    icon = folium.Icon (
        color = "blue",
        icon_color = "white",
        icon = "building ",
        prefix = "fa")
    folium.Marker([row['latitude'], row['longitude']], tooltip=tooltip, icon=icon).add_to(map_venues2)

# add layer control to toggle between the heatmaps
folium.LayerControl().add_to(map_venues2)

# display the map
map_venues2

In [17]:
map_venues3 = folium.Map(location=[possible_venues.loc[0, 'latitude'], possible_venues.loc[0, 'longitude']], zoom_start=13,tiles='cartodbpositron')
# create a list of (latitude, longitude) tuples from the grooming_venues list
grooming_locations = [(venue['lat'], venue['lon']) for venue in grooming_venues]

# create a heatmap layer with the grooming locations and add it to the map
grooming_heatmap = HeatMap(grooming_locations, name='Pet Grooming')
grooming_heatmap.add_to(map_venues3)

# create a list of (latitude, longitude) tuples from the starbucks_venues list
starbucks_locations = [(venue['lat'], venue['lon']) for venue in starbucks_venues]

# create a heatmap layer with the Starbucks locations and add it to the map
starbucks_heatmap = HeatMap(starbucks_locations, name='Starbucks')
starbucks_heatmap.add_to(map_venues3)

# create a list of (latitude, longitude) tuples from the daycare_venues list
daycare_locations = [(venue['lat'], venue['lon']) for venue in daycare_venues]

# create a heatmap layer with the Daycare locations and add it to the map
daycare_heatmap = HeatMap(daycare_locations, name='Daycare')
daycare_heatmap.add_to(map_venues3)

# create a list of (latitude, longitude) tuples from the daycare_venues list
bar_locations = [(venue['lat'], venue['lon']) for venue in bar_venues]

# create a heatmap layer with the Daycare locations and add it to the map
bar_heatmap = HeatMap(bar_locations, name='Bars')
bar_heatmap.add_to(map_venues3)

# iterate over the rows in the DataFrame and add a marker for each location
for index, row in possible_venues.iterrows():
    tooltip = f"{row['name']}: {row['address']}"
    icon = folium.Icon (
        color = "blue",
        icon_color = "white",
        icon = "building ",
        prefix = "fa")
    folium.Marker([row['latitude'], row['longitude']], tooltip=tooltip, icon=icon).add_to(map_venues3)

# add layer control to toggle between the heatmaps
folium.LayerControl().add_to(map_venues3)

map_venues3.save('image/map_allvenues.html')
# display the map
map_venues3

NameError: name 'grooming_venues' is not defined